##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Guia de início rápido para as APIs Core do TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/core/quickstart_core"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Veja em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/core/quickstart_core.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/core/quickstart_core.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte em GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/core/quickstart_core.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

Este tutorial de início rápido demonstra como você pode usar as [APIs de baixo nível do TensorFlow Core](https://www.tensorflow.org/guide/core) para criar e treinar um modelo de regressão linear múltipla que prevê a eficiência de combustível. Ele usa o dataset [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg) {:.external} que contém dados de eficiência de combustível para automóveis do final dos anos 1970 e início dos anos 1980.

Você seguirá as etapas típicas de um processo de aprendizado de máquina:

1. Carregar o dataset.
2. Construir um [pipeline de entrada](../data.ipynb) .
3. Construir um modelo de [regressão linear](https://developers.google.com/machine-learning/glossary#linear-regression) múltipla {:.external}.
4. Avaliar o desempenho do modelo.

## Configuração

Importe o TensorFlow e outras bibliotecas necessárias para começar:

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
print("TensorFlow version:", tf.__version__)
# Set a random seed for reproducible results 
tf.random.set_seed(22)

## Carga e pré-processamento do dataset

Em seguida, você precisa carregar e pré-processar o [dataset Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg) {:.external} do [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/) {:.external}. Este dataset usa uma variedade de recursos quantitativos e categóricos, como cilindros, deslocamento, potência e peso para prever a eficiência de combustível dos automóveis no final dos anos 1970 e início dos anos 1980.

O dataset contém alguns valores desconhecidos. Certifique-se de descartar todos os valores ausentes com o `pandas.DataFrame.dropna` e converter o dataset num tipo de tensor `tf.float32` com as funções `tf.convert_to_tensor` e `tf.cast` .

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

dataset = pd.read_csv(url, names=column_names, na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

dataset = dataset.dropna()
dataset_tf = tf.convert_to_tensor(dataset, dtype=tf.float32)
dataset.tail()

Em seguida, divida o dataset em conjuntos de treinamento e teste. Certifique-se de embaralhar o dataset com `tf.random.shuffle` para evitar divisões com viés.

In [ ]:
dataset_shuffled = tf.random.shuffle(dataset_tf, seed=22)
train_data, test_data = dataset_shuffled[100:], dataset_shuffled[:100]
x_train, y_train = train_data[:, 1:], train_data[:, 0]
x_test, y_test = test_data[:, 1:], test_data[:, 0]

Faça a engenharia de recursos básicos através da transformação one-hot-encoding do recurso `"Origin"` . A função `tf.one_hot` é útil para transformar esta coluna categórica em 3 colunas binárias independentes.

In [ ]:
def onehot_origin(x):
  origin = tf.cast(x[:, -1], tf.int32)
  # Use `origin - 1` to account for 1-indexed feature
  origin_oh = tf.one_hot(origin - 1, 3)
  x_ohe = tf.concat([x[:, :-1], origin_oh], axis = 1)
  return x_ohe

x_train_ohe, x_test_ohe = onehot_origin(x_train), onehot_origin(x_test)
x_train_ohe.numpy()

Este exemplo mostra um problema de regressão múltipla com preditores ou recursos em escalas bastante diferentes. Portanto, é vantajoso padronizar os dados para que cada característica tenha média zero e variância unitária. Use as funções `tf.reduce_mean` e `tf.math.reduce_std` para fazer a padronização. Você pode depois desnormalizar a previsão do modelo de regressão para obter seu valor em termos das unidades originais.

In [ ]:
class Normalize(tf.Module):
  def __init__(self, x):
    # Initialize the mean and standard deviation for normalization
    self.mean = tf.math.reduce_mean(x, axis=0)
    self.std = tf.math.reduce_std(x, axis=0)

  def norm(self, x):
    # Normalize the input
    return (x - self.mean)/self.std

  def unnorm(self, x):
    # Unnormalize the input
    return (x * self.std) + self.mean

In [ ]:
norm_x = Normalize(x_train_ohe)
norm_y = Normalize(y_train)
x_train_norm, y_train_norm = norm_x.norm(x_train_ohe), norm_y.norm(y_train)
x_test_norm, y_test_norm = norm_x.norm(x_test_ohe), norm_y.norm(y_test)

## Construção de um modelo de aprendizado de máquina

Crie um modelo de regressão linear com as APIs Core do TensorFlow. A equação para regressão linear múltipla é a seguinte:

```
onde
```

Usando o decorador `@tf.function`, o código Python correspondente será rastreado para gerar um gráfico do TensorFlow que possa ser chamado. Essa abordagem é útil para salvar e carregar o modelo após o treinamento. Ela também pode fornecer um aumento de desempenho para modelos com muitas camadas e operações complexas.

- $\underset{m\times 1}{\mathrm{Y}}$: vetor alvo
- $\underset{m\times n}{\mathrm{X}}$: matriz de atributos
- $\underset{m\times 1}{\mathrm{Y}}$: vetor de peso
- $b$: bias

Usando o decorador `@tf.function`, o código Python correspondente será rastreado para gerar um gráfico do TensorFlow que possa ser chamado. Essa abordagem é útil para salvar e carregar o modelo após o treinamento. Ela também pode fornecer um aumento de desempenho para modelos com muitas camadas e operações complexas. 

In [ ]:
class LinearRegression(tf.Module):

  def __init__(self):
    self.built = False

  @tf.function
  def __call__(self, x):
    # Initialize the model parameters on the first call
    if not self.built:
      # Randomly generate the weight vector and bias term
      rand_w = tf.random.uniform(shape=[x.shape[-1], 1])
      rand_b = tf.random.uniform(shape=[])
      self.w = tf.Variable(rand_w)
      self.b = tf.Variable(rand_b)
      self.built = True
    y = tf.add(tf.matmul(x, self.w), self.b)
    return tf.squeeze(y, axis=1)

Para cada exemplo, o modelo retorna uma previsão para o MPG do automóvel de entrada, calculando a soma ponderada de seus recursos mais um termo de bias. Essa previsão pode depois ter a padronização revertida para obter seu valor em termos das unidades originais.

In [ ]:
lin_reg = LinearRegression()
prediction = lin_reg(x_train_norm[:1])
prediction_unnorm = norm_y.unnorm(prediction)
prediction_unnorm.numpy()

## Definição de uma função de perda

Agora, defina uma função de perda para avaliar o desempenho do modelo durante o processo de treinamento.

Como os problemas de regressão lidam com saídas contínuas, o erro quadrático médio (MSE) é uma escolha ideal para a função de perda. O MSE é definido pela seguinte equação:

```
onde
```

O objetivo deste problema de regressão é encontrar o vetor de peso ideal, $w$, e bias, $b$, que minimiza a função de perda do MSE.

- $\hat{y}$: vetor de previsões
- $y$: vetor de alvos verdadeiros

O objetivo deste problema de regressão é encontrar o vetor de peso ideal, $w$, e bias, $b$, que minimiza a função de perda do MSE. 

In [ ]:
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred - y))

## Treinamento e avaliação do seu modelo

O uso de minilotes para treinamento garante eficiência de memória e convergência mais rápida. A API `tf.data.Dataset` tem funções úteis para lotes e embaralhamento. A API permite que você crie pipelines de entrada complexos a partir de peças simples e reutilizáveis. Saiba mais sobre como criar pipelines de entrada do TensorFlow [neste guia](https://www.tensorflow.org/guide/data) .

In [ ]:
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_norm, y_train_norm))
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0]).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_norm, y_test_norm))
test_dataset = test_dataset.shuffle(buffer_size=x_test.shape[0]).batch(batch_size)

Em seguida, escreva um loop de treinamento para atualizar iterativamente os parâmetros do seu modelo, fazendo uso da função de perda MSE e seus gradientes em relação aos parâmetros de entrada.

Esse método iterativo é conhecido como [método do gradiente descendente](https://developers.google.com/machine-learning/glossary#gradient-descent) {:.external}. A cada iteração, os parâmetros do modelo são atualizados dando um passo na direção oposta de seus gradientes calculados. O tamanho dessa etapa é determinado pela taxa de aprendizado, que é um hiperparâmetro configurável. Lembre-se de que o gradiente de uma função indica a direção de sua maior subida; portanto, dar um passo na direção oposta indica a direção da descida mais íngreme, o que ajuda a minimizar a função de perda do MSE.

In [ ]:
# Set training parameters
epochs = 100
learning_rate = 0.01
train_losses, test_losses = [], []

# Format training loop
for epoch in range(epochs):
  batch_losses_train, batch_losses_test = [], []

  # Iterate through the training data
  for x_batch, y_batch in train_dataset:
    with tf.GradientTape() as tape:
      y_pred_batch = lin_reg(x_batch)
      batch_loss = mse_loss(y_pred_batch, y_batch)
    # Update parameters with respect to the gradient calculations
    grads = tape.gradient(batch_loss, lin_reg.variables)
    for g,v in zip(grads, lin_reg.variables):
      v.assign_sub(learning_rate * g)
    # Keep track of batch-level training performance 
    batch_losses_train.append(batch_loss)
  
  # Iterate through the testing data
  for x_batch, y_batch in test_dataset:
    y_pred_batch = lin_reg(x_batch)
    batch_loss = mse_loss(y_pred_batch, y_batch)
    # Keep track of batch-level testing performance 
    batch_losses_test.append(batch_loss)

  # Keep track of epoch-level model performance
  train_loss = tf.reduce_mean(batch_losses_train)
  test_loss = tf.reduce_mean(batch_losses_test)
  train_losses.append(train_loss)
  test_losses.append(test_loss)
  if epoch % 10 == 0:
    print(f'Mean squared error for step {epoch}: {train_loss.numpy():0.3f}')

# Output final losses
print(f"\nFinal train loss: {train_loss:0.3f}")
print(f"Final test loss: {test_loss:0.3f}")

Desenhe as mudanças na perda de MSE ao longo do tempo. Calcular métricas de desempenho em um [conjunto de validação](https://developers.google.com/machine-learning/glossary#validation-set) designado {:.external} ou [conjunto de teste](https://developers.google.com/machine-learning/glossary#test-set) {:.external} garante que não ocorra o overfit excessivo do modelo em relação ao dataset de treinamento e possa generalizar bem para dados não vistos.

In [ ]:
matplotlib.rcParams['figure.figsize'] = [9, 6]

plt.plot(range(epochs), train_losses, label = "Training loss")
plt.plot(range(epochs), test_losses, label = "Testing loss")
plt.xlabel("Epoch")
plt.ylabel("Mean squared error loss")
plt.legend()
plt.title("MSE loss vs training iterations");

Parece que o modelo faz um bom trabalho ao ajustar os dados de treinamento e, ao mesmo tempo, generalizar bem os dados de teste não vistos.

## Salve e carregue o modelo

Comece criando um módulo de exportação que receba dados brutos e execute as seguintes operações:

- Extração de recursos
- Normalização
- Predição
- Desnormalização

In [ ]:
class ExportModule(tf.Module):
  def __init__(self, model, extract_features, norm_x, norm_y):
    # Initialize pre and postprocessing functions
    self.model = model
    self.extract_features = extract_features
    self.norm_x = norm_x
    self.norm_y = norm_y

  @tf.function(input_signature=[tf.TensorSpec(shape=[None, None], dtype=tf.float32)]) 
  def __call__(self, x):
    # Run the ExportModule for new data points
    x = self.extract_features(x)
    x = self.norm_x.norm(x)
    y = self.model(x)
    y = self.norm_y.unnorm(y)
    return y 

In [ ]:
lin_reg_export = ExportModule(model=lin_reg,
                              extract_features=onehot_origin,
                              norm_x=norm_x,
                              norm_y=norm_y)

Se você deseja salvar o modelo em seu estado atual, use a função `tf.saved_model.save`. Para carregar um modelo salvo para fazer previsões, use a função `tf.saved_model.load` .

In [ ]:
import tempfile
import os

models = tempfile.mkdtemp()
save_path = os.path.join(models, 'lin_reg_export')
tf.saved_model.save(lin_reg_export, save_path)

In [ ]:
lin_reg_loaded = tf.saved_model.load(save_path)
test_preds = lin_reg_loaded(x_test)
test_preds[:10].numpy()

## Conclusão

Parabéns! Você treinou um modelo de regressão usando as APIs de baixo nível do TensorFlow Core.

Para mais exemplos de uso das APIs do TensorFlow Core, confira os seguintes guias:

- [Regressão logística](./logistic_regression_core.ipynb) para classificação binária
- [Perceptrons multicamadas](./mlp_core.ipynb) para reconhecimento de dígitos escritos à mão
